In [ ]:
# Libraries

import random
import math
import time
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from sklearn.metrics import classification_report, accuracy_score
import xgboost
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier

In [ ]:
# Load test data

test_df = pq.read_table('tenk_test_data.pq').to_pandas()

In [ ]:
# Load train data

train_df = pq.read_table('tenk_train_data.pq').to_pandas()

In [ ]:
# Feature Engineering
# Time window segmented features

def generate_statistical_features(data_df, window_span, min_rec):
    selected_attributes = ['altitude', 'vertical_speed', 'roll', 'AOA', 'airspeed', 'flight_path_angle', 'pitch']
    input_df = data_df.iloc[:,:13].copy()    
    for attr in selected_attributes:
        input_df[attr + '_mean'] = data_df.groupby('name')[attr].transform(lambda x: x.rolling(window_span, min_rec).mean(engine='numba', raw=True)).round(3)        
        input_df[attr + '_variance'] = data_df.groupby('name')[attr].transform(lambda x: x.rolling(window_span, min_rec).var(engine='numba', raw=True)).round(3)
    
    input_df['stall'] = data_df['stall']     
    return input_df

In [ ]:
# Generating Time window segmented features train and test set

stat_train_df = generate_statistical_features(train_df, 20, 5)
stat_test_df = generate_statistical_features(test_df, 20, 5)

In [ ]:
# Drop null values

stat_train_df = stat_train_df.dropna()
stat_test_df = stat_test_df.dropna()

In [ ]:
# Train XGBoost model
# Predict
# Classification report

xgb = xgboost.XGBClassifier()
pred = xgb.fit(stat_train_df.iloc[:,13:-1], stat_train_df['stall']).predict(stat_test_df.iloc[:,13:-1])
print(classification_report(stat_test_df['stall'], pred))

In [ ]:
# Confusion Matrix

predicted_value = pred

In [ ]:
# Predicted outcome array

predicted_value

In [ ]:
# Actual values of the targeted variable in the test set

stall_column = stat_test_df.loc[:,'stall']

In [ ]:
# True/actual values

true_value = stall_column.values

In [ ]:
# True values array

true_value

In [ ]:
# Confusion Matrix

plt.rcParams["figure.figsize"] = (9,6)
target_names = ['class 0', 'class 1', 'class 2', 'class 3']
conf_matrix = confusion_matrix(true_value, predicted_value)
sns.heatmap(conf_matrix, annot=True, fmt='d'
           , xticklabels=target_names, yticklabels=target_names
           , cmap = sns.cm.rocket_r)
plt.xlabel('Predicted Label',fontsize=14)
plt.ylabel('True Label',fontsize=14)
plt.title("XGBoost Confusion Matrix",fontdict={'weight':'bold','size': 14})


In [ ]:
# Hyperparameter tunning for statistical features based model

In [ ]:
# Random Search CV

In [ ]:
# Input and Output for hyperparameter tunning and cross validation 

X = stat_train_df.iloc[:,13:-1]
y = stat_train_df['stall']

In [ ]:
# Step 1 - select some parameters that are present in the XGBClassifier

params={
    "learning_rate"   : [0.05, 0.01, 0.15, 0.20, 0.25, 0.30],
    "max_depth"       : [3, 4, 5, 6, 8, 10, 12, 15],
    "min_child_weight": [1, 3, 5, 7],
    "gamma"           : [0.0, 0.1, 0.2, 0.3, 0.4],
    "colsample_bytree": [0.3, 0.4, 0.5, 0.7]  
}

In [ ]:
# Timer

def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec =divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.'%(thour, tmin, round(tsec, 2)))

In [ ]:
# Setup default classifier

classifier = xgboost.XGBClassifier()

In [ ]:
# Set up the randomized search

random_search=RandomizedSearchCV(classifier, 
                                 param_distributions=params,
                                 n_iter=5,scoring='f1_macro', 
                                 n_jobs=-1,cv=5, 
                                 verbose=3)

In [ ]:
# Fit

start_time = timer(None) 
random_search.fit(X,y)
timer(start_time)

In [ ]:
# Best estimators

random_search.best_estimator_

In [ ]:
# Selected params from range of inputs

random_search.best_params_

In [ ]:
# Put the best parameters in

classifier = xgboost.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.25, max_delta_step=0, max_depth=5,
              min_child_weight=7, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [ ]:
# For XGBoost model with hyperparameter tuning test sets

z = stat_test_df.iloc[:,13:-1]
u = stat_test_df['stall']

In [ ]:
pred = classifier.fit(X, y).predict(z)
print(classification_report(u, pred))

In [ ]:
# Model Validation

xgb = xgboost.XGBClassifier(objective= 'multi:softmax',
                            nthread=4,
                            seed=42)

In [ ]:
# 10 fold cross validation

from sklearn.model_selection import cross_val_score
score = cross_val_score(xgb, X,y,cv=10)

In [ ]:
# Validation score

score

In [ ]:
score

In [ ]:
# Validation mean (accuracy)

score.mean()

In [ ]:
# Predicting on single simulation/flight data

df_read_one_stat = pq.read_table('single_sim_new.pq').to_pandas()

In [ ]:
# Feature Engineering for one simulation
# Time window segmentation

def generate_statistical_features(data_df, window_span, min_rec):
    selected_attributes = ['altitude', 'vertical_speed', 'roll', 'AOA', 'airspeed', 'flight_path_angle', 'pitch']
    input_df = data_df.iloc[:,:13].copy()    
    for attr in selected_attributes:
        input_df[attr + '_mean'] = data_df.groupby('name')[attr].transform(lambda x: x.rolling(window_span, min_rec).mean(engine='numba', raw=True)).round(3)        
        input_df[attr + '_variance'] = data_df.groupby('name')[attr].transform(lambda x: x.rolling(window_span, min_rec).var(engine='numba', raw=True)).round(3)
    
    input_df['stall'] = data_df['stall']     
    return input_df

In [ ]:
# Generating statistical features for one simulation

stat_one = generate_statistical_features(df_read_one_stat, 20, 5)

In [ ]:
# Drop null values

stat_one = stat_one.dropna()

In [ ]:
# XGBoost model for one simulation prediction
# Classification report on the one simulation prediction

xgb = xgboost.XGBClassifier()
pred = xgb.fit(stat_train_df.iloc[:,13:-1], stat_train_df['stall']).predict(stat_one.iloc[:,13:-1])
print(classification_report(stat_one['stall'], pred))

In [ ]:
# Prediction outcome array

pred

In [ ]:
# Prediction outcome list

list1 = pred.tolist()

print(f'List: {list1}')

In [ ]:
# Creating a data frame for the predicted outcome

df_plot = pd.DataFrame(pred)

In [ ]:
# Adding a column name

df_plot.columns =['stallp']

In [ ]:
# Predicted outcome data frame

df_plot

In [ ]:
# Adding the predicted outcome to the one simulation data

stat_one['stallp'] = df_plot['stallp'].values

In [ ]:
# One simulation prediction on the XGBoost model visualization

stall_time = stat_one.loc[stat_one['stall'] == 1, ['time']].min().values[0]
stall_time_2 = stat_one.loc[stat_one['stall'] == 2, ['time']].min().values[0]
stall_time_3 = stat_one.loc[stat_one['stall'] == 3, ['time']].min().values[0]
stall_time_4= stat_one.loc[stat_one['stallp'] == 1, ['time']].min().values[0]
stall_time_5 = stat_one.loc[stat_one['stallp'] == 2, ['time']].min().values[0]
stall_time_6 = stat_one.loc[stat_one['stallp'] == 3, ['time']].min().values[0]

In [ ]:
# Actual Onset of stall/ Class 1
stall_time

In [ ]:
# Actual Uncommanded Decent/ Class 2
stall_time_2

In [ ]:
# Actual Uncommanded Decent High & Roll/ Class 3
stall_time_3

In [ ]:
# Predicted Onset of stall/ Class 1
stall_time_4

In [ ]:
# Predicted Actual Uncommanded Decent/ Class 2
stall_time_5

In [ ]:
# Predicted Actual Uncommanded Decent High & Roll/ Class 3
stall_time_6

In [ ]:
# Latency of class 3 

69.60000000000046 - 69.30000000000047

In [ ]:
# Making strip plots 

import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
fig, axs = plt.subplots(ncols=1, nrows=7, figsize=(18,9), sharex=True)
plt = sns.lineplot(x='time', y='altitude_mean', data=stat_one, ax=axs[0])
plt.axvline(stall_time, color='red')
plt.axvline(stall_time_2, color='red')
plt.axvline(stall_time_3, color='red')
plt.axvline(stall_time_4, color='green')
plt.axvline(stall_time_5, color='green')
plt.axvline(stall_time_6, color='green')

plt = sns.lineplot(x='time', y='vertical_speed_mean', data=stat_one, ax=axs[1])
plt.axvline(stall_time, color='red')
plt.axvline(stall_time_2, color='red')
plt.axvline(stall_time_3, color='red')
plt.axvline(stall_time_4, color='green')
plt.axvline(stall_time_5, color='green')
plt.axvline(stall_time_6, color='green')

plt = sns.lineplot(x='time', y='roll_mean', data=stat_one, ax=axs[2])
plt.axvline(stall_time, color='red')
plt.axvline(stall_time_2, color='red')
plt.axvline(stall_time_3, color='red')
plt.axvline(stall_time_4, color='green')
plt.axvline(stall_time_5, color='green')
plt.axvline(stall_time_6, color='green')

plt = sns.lineplot(x='time', y='AOA_mean', data=stat_one, ax=axs[3])
plt.axvline(stall_time, color='red')
plt.axvline(stall_time_2, color='red')
plt.axvline(stall_time_3, color='red')
plt.axvline(stall_time_4, color='green')
plt.axvline(stall_time_5, color='green')
plt.axvline(stall_time_6, color='green')

plt = sns.lineplot(x='time', y='flight_path_angle_mean', data=stat_one, ax=axs[4])
plt.axvline(stall_time, color='red')
plt.axvline(stall_time_2, color='red')
plt.axvline(stall_time_3, color='red')
plt.axvline(stall_time_4, color='green')
plt.axvline(stall_time_5, color='green')
plt.axvline(stall_time_6, color='green')

plt = sns.lineplot(x='time', y='pitch_mean', data=stat_one, ax=axs[5])
plt.axvline(stall_time, color='red')
plt.axvline(stall_time_2, color='red')
plt.axvline(stall_time_3, color='red')
plt.axvline(stall_time_4, color='green')
plt.axvline(stall_time_5, color='green')
plt.axvline(stall_time_6, color='green')

plt = sns.lineplot(x='time', y='airspeed_mean', data=stat_one, ax=axs[6])
plt.axvline(stall_time, color='red')
plt.axvline(stall_time_2, color='red')
plt.axvline(stall_time_3, color='red')
plt.axvline(stall_time_4, color='green')
plt.axvline(stall_time_5, color='green')
plt.axvline(stall_time_6, color='green')

left  = 0.125  # the left side of the subplots of the figure
right = 0.9    # the right side of the subplots of the figure
bottom = 10   # the bottom of the subplots of the figure
top = 11      # the top of the subplots of the figure
wspace = 0.2   # the amount of width reserved for blank space between subplots
hspace = 0.2   # the amount of height reserved for white space between subplots

In [ ]:
# To visualize actual vs predicted scatterplots

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

labels_dict = {0: "Level Flight", 1: "Onset of Stall", 
                   2: "Uncommanded Descent", 3: "Uncommanded Roll/DescentHigh"}

flight_data = ptest_df[ptest_df['name']==flight_name] # Assign a flight_name

x = flight_data['time']
y = flight_data['stall'].replace(labels_dict)
y1 = flight_pred # Assign output from the classifier for this particular flight

sns.set(style="darkgrid")
plt.figure(figsize=(20,4))
plt.scatter(x, y, label = "Actual", s=100)
plt.scatter(x, y1, label = "Predicted")
plt.xlabel('Time (sec)')
plt.legend(loc="lower right")
fig = plt.gcf()
plt.show()

In [ ]:
# Explainability

import lime
import lime.lime_tabular

explainer = lime.lime_tabular.LimeTabularExplainer(ptrain_df.iloc[:,13:-1].values, 
                                                   feature_names=ptrain_df.columns.tolist()[13:-1], 
                                                   training_labels=ptrain_df['stall'], 
                                                   verbose=True, mode='classification', 
                                                   class_names = [0, 1, 2, 3])

exp = explainer.explain_instance(ptest_df.loc[index, ptest_df.columns.tolist()[13:-1]], 
                                 xgbmodel.predict_proba, num_features=5, top_labels=5)
exp.show_in_notebook(show_table=True)

In [ ]:
# Method 2
# XGBoost using Lag features

In [ ]:
# Input: Raw data (df) with preferred lag range values 
# Output: Dataframe with Lagged Features

def generate_lagged_feature(data_df, start_lag, end_lag):
    selected_attributes = ['altitude', 'vertical_speed', 'roll', 'AOA', 'airspeed', 'flight_path_angle', 'pitch']
    input_df = data_df.iloc[:,:-1].copy()
    for attr in selected_attributes:
        for lag_i in range(start_lag, end_lag, 25):
            col_name = attr + '_lag_' + str(lag_i)
            input_df[col_name] = input_df.groupby('name')[attr].shift(lag_i)
    input_df['stall'] = data_df['stall']       
    return input_df

In [ ]:
# Generating Lagged features for training and test sets

lagged_train_df = generate_lagged_feature(train_df, 25, 150)
lagged_test_df = generate_lagged_feature(test_df,25, 150)

In [ ]:
# Drop null vales

lagged_train_df = lagged_train_df.dropna()
lagged_test_df = lagged_test_df.dropna()

In [ ]:
# XGBoost for lagged method
# Model training and prediction
# Classification report of XGBoost model based on lagged features 

xgb = xgboost.XGBClassifier(objective= 'multi:softmax',
                            nthread=4,
                            seed=42)
pred = xgb.fit(lagged_train_df.iloc[:,13:-1], lagged_train_df['stall']).predict(lagged_test_df.iloc[:,13:-1])
print(classification_report(lagged_test_df['stall'], pred))

In [ ]:
xgb = xgboost.XGBClassifier()
pred = xgb.fit(lagged_train_df.iloc[:,13:-1], lagged_train_df['stall']).predict(lagged_test_df.iloc[:,13:-1])
print(classification_report(lagged_test_df['stall'], pred))

In [ ]:
# Single simulation data

df_read_one = pq.read_table('single_sim_new.pq').to_pandas()

In [ ]:
# Input: Raw data (df) with preferred lag range values 
# Output: Dataframe with Lagged Features

def generate_lagged_feature(data_df, start_lag, end_lag):
    selected_attributes = ['altitude', 'vertical_speed', 'roll', 'AOA', 'airspeed', 'flight_path_angle', 'pitch']
    input_df = data_df.iloc[:,:-1].copy()
    for attr in selected_attributes:
        for lag_i in range(start_lag, end_lag, 25):
            col_name = attr + '_lag_' + str(lag_i)
            input_df[col_name] = input_df.groupby('name')[attr].shift(lag_i)
    input_df['stall'] = data_df['stall']       
    return input_df

In [ ]:
# Generating Lagged features for one simulation prediction on XGBoost model

lagged_one = generate_lagged_feature(df_read_one, 25, 150)

In [ ]:
# Drop null values

lagged_one = lagged_one.dropna()

In [ ]:
# One simulation prediction on the Lagged features based XGBoost model

xgb = xgboost.XGBClassifier()
pred = xgb.fit(lagged_train_df.iloc[:,13:-1], lagged_train_df['stall']).predict(lagged_one.iloc[:,13:-1])
print(classification_report(lagged_one['stall'], pred))

In [ ]:
# One simulation with lagged features prediction outcome

pred

In [ ]:
list1 = pred.tolist()

print(f'List: {list1}')

In [ ]:
# Dataframe of the predicted outcome

df_plot = pd.DataFrame(pred)

In [ ]:
# Add column name

df_plot.columns =['stallp']

In [ ]:
# Combine the prediction output to the actual data

lagged_one['stallp'] = df_plot['stallp'].values

In [ ]:
# One simulation prediction plot

stall_time = lagged_one.loc[lagged_one['stall'] == 1, ['time']].min().values[0]
stall_time_2 = lagged_one.loc[lagged_one['stall'] == 2, ['time']].min().values[0]
stall_time_3 = lagged_one.loc[lagged_one['stall'] == 3, ['time']].min().values[0]
stall_time_4= lagged_one.loc[lagged_one['stallp'] == 1, ['time']].min().values[0]
stall_time_5 = lagged_one.loc[lagged_one['stallp'] == 2, ['time']].min().values[0]
stall_time_6 = lagged_one.loc[lagged_one['stallp'] == 3, ['time']].min().values[0]

In [ ]:
# Class 1 - Onset of stall actual

stall_time 

In [ ]:
# Class 2 - Uncommanded Decent actual

stall_time_2 

In [ ]:
# Class 3 - Uncommanded Decent High and Roll actual

stall_time_3 

In [ ]:
# Class 1 - Onset of stall predicted

stall_time_4 - Onset of stall predicted

In [ ]:
# Class 2 - Uncommanded Decent predicted

stall_time_5

In [ ]:
# Class 3 - Uncommanded Decent High and Roll predicted

stall_time_6

In [ ]:
# Latency class 1

50.1 - 52

In [ ]:
# Latency class 3

69.2 - 69.3

In [ ]:
## Make plots 

import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
fig, axs = plt.subplots(ncols=1, nrows=7, figsize=(18,9), sharex=True)
plt = sns.lineplot(x='time', y='altitude', data=lagged_one, ax=axs[0])
plt.axvline(stall_time, color='red')
plt.axvline(stall_time_2, color='red')
plt.axvline(stall_time_3, color='red')
plt.axvline(stall_time_4, color='green')
plt.axvline(stall_time_5, color='green')
plt.axvline(stall_time_6, color='green')

plt = sns.lineplot(x='time', y='vertical_speed', data=lagged_one, ax=axs[1])
plt.axvline(stall_time, color='red')
plt.axvline(stall_time_2, color='red')
plt.axvline(stall_time_3, color='red')
plt.axvline(stall_time_4, color='green')
plt.axvline(stall_time_5, color='green')
plt.axvline(stall_time_6, color='green')

plt = sns.lineplot(x='time', y='roll', data=lagged_one, ax=axs[2])
plt.axvline(stall_time, color='red')
plt.axvline(stall_time_2, color='red')
plt.axvline(stall_time_3, color='red')
plt.axvline(stall_time_4, color='green')
plt.axvline(stall_time_5, color='green')
plt.axvline(stall_time_6, color='green')

plt = sns.lineplot(x='time', y='AOA', data=lagged_one, ax=axs[3])
plt.axvline(stall_time, color='red')
plt.axvline(stall_time_2, color='red')
plt.axvline(stall_time_3, color='red')
plt.axvline(stall_time_4, color='green')
plt.axvline(stall_time_5, color='green')
plt.axvline(stall_time_6, color='green')

plt = sns.lineplot(x='time', y='flight_path_angle', data=lagged_one, ax=axs[4])
plt.axvline(stall_time, color='red')
plt.axvline(stall_time_2, color='red')
plt.axvline(stall_time_3, color='red')
plt.axvline(stall_time_4, color='green')
plt.axvline(stall_time_5, color='green')
plt.axvline(stall_time_6, color='green')

plt = sns.lineplot(x='time', y='pitch', data=lagged_one, ax=axs[5])
plt.axvline(stall_time, color='red')
plt.axvline(stall_time_2, color='red')
plt.axvline(stall_time_3, color='red')
plt.axvline(stall_time_4, color='green')
plt.axvline(stall_time_5, color='green')
plt.axvline(stall_time_6, color='green')

plt = sns.lineplot(x='time', y='airspeed', data=lagged_one, ax=axs[6])
plt.axvline(stall_time, color='red')
plt.axvline(stall_time_2, color='red')
plt.axvline(stall_time_3, color='red')
plt.axvline(stall_time_4, color='green')
plt.axvline(stall_time_5, color='green')
plt.axvline(stall_time_6, color='green')

left  = 0.125  # the left side of the subplots of the figure
right = 0.9    # the right side of the subplots of the figure
bottom = 10   # the bottom of the subplots of the figure
top = 11      # the top of the subplots of the figure
wspace = 0.2   # the amount of width reserved for blank space between subplots
hspace = 0.2   # the amount of height reserved for white space between subplots

In [ ]:
# Hyperparameter tuning for Lagged features based model

In [ ]:
X = lagged_train_df.iloc[:,13:-1]
y = lagged_train_df['stall']

In [ ]:
# Step 1 - select some parameters that are present in the XGBClassifier

params={
    "learning_rate"   : [0.05, 0.01, 0.15, 0.20, 0.25, 0.30],
    "max_depth"       : [3, 4, 5, 6, 8, 10, 12, 15],
    "min_child_weight": [1, 3, 5, 7],
    "gamma"           : [0.0, 0.1, 0.2, 0.3, 0.4],
    "colsample_bytree": [0.3, 0.4, 0.5, 0.7]
    
}

In [ ]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec =divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.'%(thour, tmin, round(tsec, 2)))

In [ ]:
# Setup default classifier

classifier = xgboost.XGBClassifier()

In [ ]:
# Fit

start_time = timer(None) 
random_search.fit(X,y)
timer(start_time)

In [ ]:
# Best estimators

random_search.best_estimator_

In [ ]:
# Params that were selected

random_search.best_params_

In [ ]:
# Putting the best parameters in

classifier = xgboost.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.25, max_delta_step=0, max_depth=5,
              min_child_weight=7, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [ ]:
# Cross validation on the lagged featureds based model

from sklearn.model_selection import cross_val_score
score = cross_val_score(classifier, X,y,cv=10)

In [ ]:
# Validation score

score

In [ ]:
# Accuracy

score.mean()

In [ ]:
# For Lagged feature based model with hyperparameter tuning

z = lagged_test_df.iloc[:,13:-1]
u = lagged_test_df['stall']

In [ ]:
# Lagged feature based model with hyperparameter tuning and classification report

pred = classifier.fit(X, y).predict(z)
print(classification_report(u, pred))